In [230]:
# Specifying all the imports
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.prompts.prompt import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.agents import create_react_agent, AgentExecutor
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from dotenv import load_dotenv
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.exceptions import OutputParserException
import re
from langchain_core.prompts.few_shot import FewShotPromptTemplate
import os

In [231]:
# Loading dotenv and specifying tools
load_dotenv()

def extract_clients(query: str) -> list:
    num_list = []
    if 'all clients' in query:
        for i in range(1,11):
            num_list.append(i)
    else:
        for i in query:
            if i.isdigit():
                num_list.append(int(i))
    return num_list

        

@tool
def search_tool(query : str):
    """ 
    Takes a user query and performs an online search to provide
    an answer to the query
    """
    
    search = DuckDuckGoSearchRun()
    answer = search.run(query)
    return answer

@tool
def create_embeddings(query : str):
    """
    Uses the content from the file path to create embeddings on the inputted data. Using the query
    provided, perform a similarity search on the query and return the output. If you can not provide a response,
    say I don't know rather than providing an incorrect response.
    """

    #file_path = r'C:\GenAI\Final Project\GenAIGroupProject\apple.txt'
    file_path = r'./apple.txt'
    file_path = file_path.strip()
    loader = TextLoader(file_path, encoding= 'utf-8')
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunk = text_splitter.split_documents(documents=document)
    embeddings = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'))
    vectorstore = FAISS.from_documents(chunk, embeddings)

    docs = vectorstore.similarity_search(query)
    return docs[0].page_content


@tool
def query_database(query : str):
    """
    This function queries the SQL database based on the query that the user provides
    to provide insight into the current client stock portfolio
    """

    

    # Initialize the JSON output key parser


    llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0)
    db = SQLDatabase.from_uri('sqlite:///portfolio_allocations.db')
    agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
    result = agent_executor.invoke(query)

    
    clients = [f"client{i}" for i in extract_clients(query)]

    # Define the response schemas dynamically based on the number of clients
    response_schemas = [
        ResponseSchema(name=client, description=f"A dictionary containing stock tickers and their percentage allocations for {client}.")
        for client in clients
    ]

    # Initialize the structured output parser
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

    # Get the format instructions
    format_instructions = output_parser.get_format_instructions()

    # Define the prompt template
    prompt_template = PromptTemplate(
        template="Create a new key, value pair for every new client mentioned in the question.\n{format_instructions}\nQuestion: {question}",
        input_variables=["question"],
        partial_variables={"format_instructions": format_instructions},
    )

    
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

    


    format_instructions = output_parser.get_format_instructions()
    
    prompt = PromptTemplate(
    template="create a new key, value pair for every new client mentioned in the question.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

    
    
    chain = prompt | llm | output_parser

    ans = chain.invoke({"question": result})

    return ans

    
@tool
def portfolio_allocation(query : str):
    """
    Based on the allocation stratergy provided as a query, change the allocation in the
    client stock portfolio in the SQL database
    """
    #query = Reallocte the portfolio for client 1 with a balanced strategy
    llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0)

    examples = [
        {
            'query' : 'Reallocte the portfolio for client 1 with a balanced strategy',
            'answer' : ' The stock allocations for client_id 1 to 33,33,33'
        },

        {
            'query' : 'Reallocte the portfolio for client 1 with a risk-based strategy',
            'answer' : """
                        Allocating more to stocks with lower perceived risk. 
                        From my 10k data and online search, AAPL has strong financialas
                        and minimal risk but NVIDIA has more risk. As a result, the stock
                        allocation fror client_id 1 would have a high percentage for AAPL
                        and a low percentage for NVIDIA.
                        """
        },

        {
            'query': 'Reallocte the portfolio for client 1 with a return-based strategy',
            'answer' : """
                        Allocating more to stocks with higher expected returns.
                        From my 10k data and online search, NVIDIA has higher expected growth rates and returns
                        whereas MSFT has a declining industry position and poor management quality. As a result, the stock
                        allocation fror client_id 1 would have a high percentage for NVIDIA
                        and a low percentage for MSFT
                        """
        }
    ]
    
    
    
    ten_k_data = create_embeddings('Summarize the current financial situation of AAPL, MSFT & NVIDIA')
    online_search = search_tool('Summarize the current financial situation of AAPL, MSFT & NVIDIA')

    example_template = """
    User: {query}
    AI: {answer}
    """

    example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
    )

    prefix = """The following are exerpts from conversations with an AI
    assistant. The assistant uses the context provided from the 10k and online search to gather information
    about the company's financial situation using the create_embeddings and search_tool.
    to make informed decisions based on the investment strategy in the user query.
    The assistant then reallocated the user's stock portfolio. This should be done only
    for AAPL, MSFT, NVIDIA.
    """

    suffix = """
    User: {query}
    AI: 
    """

    few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
    )

    chain = few_shot_prompt_template | llm

    ans = chain.invoke(query)
    return ans

       

    #  1. Read in the client database using the query database tool
    #  = f"Return the values of the stock holdings for client_id {query} in the sql database"
    #query_database(s1_query)
    
    #  2. Determine allocation stratergy based on query
    
    

    #  3. Read in the Financials from the 10k

    #  4. Get current information from online

    #  5. Apply the stratergy we determined on the information from steps 3 and 4!

    #  6. Translate this into a percentage allocation for each company and write to database

    #  7. Update database based on new percentages



    

tools = [Tool(name="search_tool", func=search_tool, description="Tool for performing online search operations"),
         Tool(name="create_embeddings", func=create_embeddings, description="Uses the content from the file path to create embeddings on the inputted data. Using the query provided, perform a similarity search on the query and return the output. If you can not provide a response,say I don't know rather than providing an incorrect response."),
        Tool(name='query_database', func=query_database, description= "This function queries the SQL database based on the query that the user provides to provide insight into the current client stock portfolio"),
        Tool(name='portfolio_allocation', func=portfolio_allocation, description = 'This function rellocated stocks for the specified clients based on context and their investment stratergy')]

In [232]:
template = """
You need to use the tools to find the appropriate action to take. 
Only use the query_database and portfolio_allocation tools when the term "clients" is mentioned in the query.
If further queries are presented alongside the term "clients", then use all tools available.
Otherwise, ignore these two tools and use the embeddings first
to see if you can get relevant responses from the similarity search from the user's query. If you do get relevant
responses, please return the responses from the similarity search. If you can't get relevent results from the embeddings,
use the seatch tool and get the reponse from there. If you still can't find an answer from an online search, please do not make up an
answer and return 'Sorry I can not answer your query' instead.

If performing portfolio_allocation, get context of the company's financial situation by using the create_embeddings and search tool by querying:
"Tell me about the company's financial situation" and make the percentage splits based on this and the investment stratergy. This should be done only
for AAPL, MSFT, NVIDIA. Give the response by including actual percentages rather than 'higher' or 'lower' percentage.

You have access to the following tools:
{tools}

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

If only the query_database is used, return  what is in the function as a dictionary as the final answer. It
shoudl be in this format:



Question: {input}

Thought:{agent_scratchpad}
"""

# Define the query and create the prompt
#query = 'return the values for client_id 1 in the sql database and also tell me the current apple stock price'
#query = 'Tell me the current apple stock price'
query = 'Reallocte the portfolio for client 6 with a return-based strategy'
#query = "The Company continues to develop new technologies to enhance existing products and services and to expand the range of its offerings through research and development RD licensing of intellectual property and acquisition of thirdparty businesses and technology"
#query = "Give me a sentence from the apple 10-k report"
#query = "what is the net sales of Macs in 2021"
#query =  "add a new client to the database with random stock allocations, provide the sql query to do so and execute it"
#query =  "set the stock allocations for client_id 1 to 5,5,90. provide the sql query to do so and execute it on the database" #remeber to pull numbers from another query into this framework

prompt = PromptTemplate(input_variables=['input','tools', 'agent_scratchpad', 'tool_names'], template=template)

# Initialize the language model
llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0)


# Create the agent using the language model and the toolset
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=False)

# Execute the agent with the given input
response = agent_executor.invoke({'input': query, 'tools' : tools, 'tool_names' : [tool.name for tool in tools]})

print(response['output'])



> Entering new AgentExecutor chain...
I should use the portfolio_allocation tool to reallocate the portfolio for client 6 based on a return-based strategy.

Action: portfolio_allocation

Action Input: 
{
  "client_id": 6,
  "strategy": "return-based"
}
content='Allocating more to stocks with higher expected returns.\nFrom my 10k data and online search, NVIDIA has higher expected growth rates and returns\nwhereas MSFT has a declining industry position and poor management quality. As a result, the stock\nallocation for client_id 6 would have a high percentage for NVIDIA\nand a low percentage for MSFT.' response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 362, 'total_tokens': 434}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4bc63fb4-4c78-41fd-bf72-bdbe8a4add5a-0'Final Answer: {
    "client_id": 6,
    "strategy": "return-based",
    "allocation": {
        "NVIDIA": 70,
        "MSFT": 30
    }
}
